In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Flatten, Dense, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVC, SVR, LinearSVC, LinearSVR

from tensorflow.keras.optimizers import SGD, Adam
import matplotlib .pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score

from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob
import math

from tqdm import tqdm_notebook
from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.applications import InceptionV3, MobileNet

In [2]:
import matplotlib.font_manager as fm # 폰트연결 시켜주는 모듈
font_name = fm.FontProperties(fname = "C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family = font_name)

# 마이너스 사인 해결
import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

# 워닝소리 미전시
import warnings
warnings.filterwarnings("ignore") 

In [3]:
farm1_outer = pd.read_csv("C:/YOONJINSEOK/Python work/AI/project/tomato_farm_set/tom1.csv", encoding = "utf-8", index_col = 0)
round(farm1_outer.isnull().sum()/len(farm1_outer)*100, 2) # 총 3438개

날짜           0.00
J/Day        0.00
평균온도        86.59
EC급액        86.59
EC배액        86.59
pH급액        86.59
pH배액        86.59
내부이산화탄소     86.59
온도토양       100.00
온도급액       100.00
온도배액       100.00
dtype: float64

In [4]:
farm1 = pd.read_csv("C:/YOONJINSEOK/Python work/AI/project/tomato_farm_set/tom1_inner.csv", encoding = "utf-8", index_col = 0)
farm1.isnull().sum() # 총 461개

farm2 = pd.read_csv("C:/YOONJINSEOK/Python work/AI/project/tomato_farm_set/tom2.csv", encoding = "utf-8", index_col = 0)
round(farm2.isnull().sum()/len(farm2)*100, 2) # 총 1359개

farm3 = pd.read_csv("C:/YOONJINSEOK/Python work/AI/project/tomato_farm_set/tom3.csv", encoding = "utf-8", index_col = 0)
round(farm3.isnull().sum()/len(farm3)*100, 2) # 총 18888개 / 버린다.

farm4 = pd.read_csv("C:/YOONJINSEOK/Python work/AI/project/tomato_farm_set/tom4.csv", encoding = "utf-8", index_col = 0)
round(farm4.isnull().sum()/len(farm4)*100, 2) # 2799개 / 버리자

farm5 = pd.read_csv("C:/YOONJINSEOK/Python work/AI/project/tomato_farm_set/tom5.csv", encoding = "utf-8", index_col = 0)
round(farm5.isnull().sum()/len(farm5)*100, 2) # 총 119개

farm6 = pd.read_csv("C:/YOONJINSEOK/Python work/AI/project/tomato_farm_set/tom6.csv", encoding = "utf-8", index_col = 0)
round(farm6.isnull().sum()/len(farm6)*100, 2) # 총 1370개

날짜          0.00
J/Day      15.33
평균온도       67.81
EC급액       67.81
EC배액       67.81
pH급액       67.81
pH배액       67.81
내부이산화탄소    67.81
온도토양       67.81
온도급액       67.81
온도배액       67.81
dtype: float64

In [5]:
root = "C:/YOONJINSEOK/Python work/AI/project/tomato_farm_set/"

In [6]:
# 21년도 파괴데이터
dst_tom_21 = []
for i in range(45, 53):
    try:
        dst_tom_df = pd.read_excel(root + "destroy_tomato.xlsx", 
                      sheet_name = "{}주차".format(i), 
                      skiprows = 3)
        print("{}주차".format(i), dst_tom_df.head())
        dst_tom_df = dst_tom_df.loc[1:100]
        dst_tom_df["주차"] = "{}".format(i)
        dst_tom_21.append(dst_tom_df[["주차", "과중", "당도", "산도"]])          
        
    except Exception as e: 
        print(e)
        
# 22년도 파괴데이터
dst_tom_22 = []
for i in range(1, 7):
    try:    
        dst_tom_df = pd.read_excel(root + "destroy_tomato.xlsx", 
                        sheet_name = "{}주차".format(str(i).zfill(2)),
                        skiprows = 3)
        print("{}주차".format(i).zfill(2), dst_tom_df)
        dst_tom_df = dst_tom_df.loc[1:100]
        dst_tom_df["주차"] = "{}".format(str(i).zfill(2))
        dst_tom_22.append(dst_tom_df[["주차", "과중", "당도", "산도"]])
            
    except Exception as e: 
        print(e)

45주차    수집항목   과폭   과고   과중      당도    산도    경도    수분율
0  개체번호   cm   cm    g  Brix %  0-14   kgf      %
1     1  7.4  5.8  205     4.1     4  2.65  71.25
2     2  7.9  6.2  210     4.7     4   2.7   72.5
3     3  7.7    6  205     4.7     4   2.7  71.25
4     4  7.8  5.7  206     4.7   4.5  2.72     75
46주차    수집항목   과폭   과고   과중      당도    산도    경도    수분율
0  개체번호   cm   cm    g  Brix %  0-14   kgf      %
1     1  8.6  6.6  276     4.2     4  2.65   76.5
2     2  9.2    7  375     4.1     4   2.7   72.5
3     3  7.6  7.1  241     4.2     4   2.7  71.25
4     4  8.4  6.7  247     3.8     4  2.72     75
47주차    수집항목   과폭   과고   과중      당도    산도    경도    수분율
0  개체번호   cm   cm    g  Brix %  0-14   kgf      %
1     1  6.6  6.9  168       4     4  2.65   76.5
2     2  7.5    7  223       4     4  2.71   72.5
3     3    7  6.5  182       4     4   2.8  71.25
4     4  6.4    7  159     3.7     4  2.72   79.5
48주차    수집항목    과폭   과고   과중      당도    산도    경도   수분율
0  개체번호    cm   cm    g  Brix 

In [7]:
# 원본데이터 합체
tom_df = pd.concat([farm1, farm2], axis = 0)
tom_df = pd.concat([tom_df, farm5], axis = 0)
tom_df = pd.concat([tom_df, farm6], axis = 0)

In [8]:
tom_df["날짜"] = pd.to_datetime(tom_df["날짜"])

In [9]:
## 환경데이터와 파괴데이터 결합
# 환경데이터 "주차" 컬럼 추가
week_list = []
for i in tom_df["날짜"]:
    b = pd.to_datetime("2021-01-01")
    weeks = math.trunc((i - b).days / 7)
    if weeks <= 52:
        week_list.append(str(weeks).zfill(2))
    elif weeks > 52:
        week_list.append(str(weeks - 52).zfill(2))
        
tom_df["주차"] = week_list # 주차 컬럼 추가
tom_df["주차"] = tom_df["주차"].astype(int) # 주차 자료형 변경
tom_df["주차"].value_counts().sort_index() 

1     166
2     168
3     167
4      96
41     14
42    172
43    203
44    192
45    309
46    368
47    352
48    203
49    335
50    278
51    118
52    168
Name: 주차, dtype: int64

In [10]:
## 45 ~ 52주차 데이터 처리(21년)

import random
np.random.seed(10)

data_21 = []
for i in range(len(dst_tom_21)):
    df_week = tom_df[tom_df["주차"] == (i + 45)]
    
    
    # 환경데이터 중 주차별 100개 임의추출
    idx_list = random.sample(range(len(df_week)), 100)
    df_week = df_week.iloc[idx_list]
    
    # 훈련데이터 
    X = df_week[df_week.columns[1:-1]] 
    X.index = np.arange(1, 101)
    
    # 정답데이터
    y = dst_tom_21[i][dst_tom_21[i].columns[1:]] 
    
    # 전체데이터 set
    data_21.append(pd.concat([X, y], axis = 1))
    
## 1 ~ 3주차 데이터 처리(22년)
import random

data_22 = []
for i in range(len(dst_tom_22)):
    try:
        df_week = tom_df[tom_df["주차"] == (i+1)]

        # 환경데이터 중 주차별 100개 임의추출
        idx_list = random.sample(range(len(df_week)), 100)
        df_week = df_week.iloc[idx_list]

        # 훈련데이터 
        X = df_week[df_week.columns[1:-1]] 
        X.index = np.arange(1, 101)

        # 정답데이터
        y = dst_tom_22[i][dst_tom_22[i].columns[1:]] 

        # 전체데이터 set
        data_22.append(pd.concat([X, y], axis = 1))
    except : None
    
### 4주차 별도처리
df_week = tom_df[tom_df["주차"] == 4]

# 4주차 파괴데이터 100개 중 shuffle 후 96개 데이터 임의추출
idx = np.arange(1, 101)
np.random.shuffle(idx)
dst_tom_22[3].index = idx
dst_tom_22[3] = dst_tom_22[3].iloc[0:96]
dst_tom_22[3].index = np.arange(1, 97)

# 훈련데이터 
X = df_week[df_week.columns[1:-1]] 
X.index = np.arange(1, 97)

# 정답데이터
y = dst_tom_22[3][dst_tom_22[3].columns[1:]] 

# 전체데이터 셋 맞추기
data_22.append(pd.concat([X, y], axis = 1))

In [11]:
df1 = pd.concat(data_21, axis = 0)
df2 = pd.concat(data_22, axis = 0)
tom_origin = pd.concat([df1, df2], axis = 0)

In [12]:
# 1) EC급액, EC배액, CO2, PH급액, ph배액
tom_origin.loc[tom_origin["EC급액"] >= 3.5, "EC급액"] = None
tom_origin.loc[tom_origin["EC배액"] >= 5.0, "EC배액"] = None
tom_origin.loc[(tom_origin["내부이산화탄소"] >= 600) | (tom_origin["내부이산화탄소"] <= 400), "내부이산화탄소"] = None
tom_origin.loc[(tom_origin["pH급액"] >= 6.5) | (tom_origin["pH급액"] <= 5.0), "pH급액"] = None
tom_origin.loc[(tom_origin["pH배액"] >= 7) | (tom_origin["pH급액"] <= 5.0), "pH급액"] = None

In [13]:
# 2) J/Day, 온도토양, 온도급액, 온도배액

# J/Day : "0"값 15개 null값으로 변경
# tom_origin["J/Day"].value_counts().sort_index()
tom_origin.loc[tom_origin["J/Day"] < 600, "J/Day"] = None

# 온도토양 : "10도 이하" null값으로 변경
# tom_origin["온도토양"].value_counts().sort_index()
tom_origin.loc[(tom_origin["온도토양"] <= 15) | (tom_origin["온도토양"] >= 30), "온도토양"] = None

# 평균온도 : "10도이하, 30도 이상" null값으로 변경
tom_origin.loc[(tom_origin["평균온도"] <= 15) | (tom_origin["평균온도"] >= 30), "평균온도"] = None

# 온도급액 : "10도이하"  null값으로 변경
# tom_origin["온도급액"].value_counts().sort_index()
tom_origin.loc[(tom_origin["온도급액"] <= 15) | (tom_origin["온도급액"] >= 30), "온도급액"] = None

# 온도배액 : "10도이하"  null값으로 변경
# tom_origin["온도배액"].value_counts().sort_index()
tom_origin.loc[(tom_origin["온도배액"] <= 15) | (tom_origin["온도배액"] >= 30) , "온도배액"] = None

In [14]:
# 3) 과중, 당도, 산도 

# 과중 : 변동없음 유지
# tom_origin["과중"].value_counts().sort_index(ascending = False)

# 당도 : "44.5" -> "4.5" 변경 / 콜라 당도 11임을 고려 존재 할 수 없는 토마토
# tom_origin["당도"].value_counts().sort_index(ascending = False)
tom_origin.loc[tom_origin["당도"] == tom_origin["당도"].max(), "당도"] = 4.5

# 산도 : 변동없음 유지
# tom_origin["산도"].value_counts().sort_index(ascending = False)

In [15]:
tom_origin

,EC급액,EC배액,pH급액,pH배액,내부이산화탄소,평균온도,J/Day,온도토양,온도급액,온도배액,과중,당도,산도
1,NaN,NaN,NaN,NaN,NaN,NaN,1499.0,NaN,NaN,NaN,205,4.1,4
2,NaN,NaN,NaN,NaN,NaN,NaN,1560.0,NaN,NaN,NaN,210,4.7,4
3,NaN,NaN,5.660515,5.894754,553.805970,17.119403,1199.0,NaN,NaN,NaN,205,4.7,4
4,NaN,NaN,NaN,NaN,NaN,NaN,1412.0,NaN,NaN,NaN,206,4.7,4.5
5,NaN,NaN,5.676011,5.795994,504.994318,27.460341,NaN,NaN,NaN,NaN,223,4.3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,2.444833,3.705833,5.606667,6.172000,NaN,NaN,NaN,20.227000,NaN,NaN,148,4.5,4
93,2.412500,3.646833,5.630167,6.194500,NaN,NaN,NaN,19.312000,NaN,NaN,142,4.6,4
94,2.400000,3.621500,5.631667,6.206333,NaN,NaN,NaN,18.660667,NaN,NaN,151,4.5,3.5
95,2.392500,3.611667,5.647167,6.193000,NaN,NaN,NaN,18.066833,NaN,NaN,145,4.3,3.5


In [16]:
# 정규화
X = tom_origin[tom_origin.columns[:-3]]
y = tom_origin[tom_origin.columns[-3:]]
scaler = StandardScaler()
tom_ori_X_scaler = scaler.fit_transform(X)
tom_ori_X_scaler = pd.DataFrame(tom_ori_X_scaler, columns = tom_origin.columns[:-3])

y = y.reset_index()
del y["index"]

In [17]:
tom_ori_scaler = pd.concat([tom_ori_X_scaler, y], axis = 1)
tom_ori_scaler

,EC급액,EC배액,pH급액,pH배액,내부이산화탄소,평균온도,J/Day,온도토양,온도급액,온도배액,과중,당도,산도
0,NaN,NaN,NaN,NaN,NaN,NaN,1.319665,NaN,NaN,NaN,205,4.1,4
1,NaN,NaN,NaN,NaN,NaN,NaN,1.592903,NaN,NaN,NaN,210,4.7,4
2,NaN,NaN,0.007192,-1.134316,2.264434,-0.818728,-0.024132,NaN,NaN,NaN,205,4.7,4
3,NaN,NaN,NaN,NaN,NaN,NaN,0.929964,NaN,NaN,NaN,206,4.7,4.5
4,NaN,NaN,0.090350,-1.769745,1.187481,2.599564,NaN,NaN,NaN,NaN,223,4.3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,-1.363659,-0.001634,-0.281773,0.649519,NaN,NaN,NaN,1.369575,NaN,NaN,148,4.5,4
1192,-1.490050,-0.116240,-0.155665,0.794286,NaN,NaN,NaN,0.853160,NaN,NaN,142,4.6,4
1193,-1.538912,-0.165449,-0.147616,0.870423,NaN,NaN,NaN,0.485556,NaN,NaN,151,4.5,3.5
1194,-1.568229,-0.184550,-0.064439,0.784635,NaN,NaN,NaN,0.150404,NaN,NaN,145,4.3,3.5


In [18]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer(max_iter=10000, random_state = 0)
tom_mul = imputer.fit_transform(tom_origin)

# numpy를 DataFrame으로 변환 / 컬럼 재설정
tom_mul = pd.DataFrame(tom_mul)
tom_mul.columns = ['EC급액', 'EC배액', 'pH급액', 'pH배액', '내부이산화탄소', '평균온도', 'J/Day', '온도토양',
   '온도급액', '온도배액', '과중', '당도', '산도']
tom_mul.head()

,EC급액,EC배액,pH급액,pH배액,내부이산화탄소,평균온도,J/Day,온도토양,온도급액,온도배액,과중,당도,산도
0,2.876224,3.596165,5.682269,6.034272,433.472317,19.322963,1499.000000,17.682453,19.214050,19.631269,205.0,4.1,4.0
1,2.879453,3.502419,5.696574,6.020120,403.647635,19.679797,1560.000000,17.685968,19.482957,19.937158,210.0,4.7,4.0
2,2.784352,3.622177,5.660515,5.894754,553.805970,17.119403,1199.000000,17.685057,17.726729,18.103952,205.0,4.7,4.0
3,2.851406,3.603188,5.680561,6.048260,428.710361,19.725948,1412.000000,18.042475,19.559605,19.986085,206.0,4.7,4.5
4,2.946274,4.268803,5.676011,5.795994,504.994318,27.460341,487.173126,20.118731,23.217148,23.307148,223.0,4.3,4.0


In [19]:
tom_mul["EC비율"] = (tom_mul["EC배액"] / tom_mul["EC급액"])
tom_mul["pH비율"] = (tom_mul["pH배액"] / tom_mul["pH급액"])
tom_mul = tom_mul[['EC급액', 'EC배액', 'pH급액', 'pH배액', '내부이산화탄소', '평균온도', 'J/Day', '온도토양',
       '온도급액', '온도배액', 'EC비율', 'pH비율', '과중', '당도', '산도']]
tom_mul.head()

,EC급액,EC배액,pH급액,pH배액,내부이산화탄소,평균온도,J/Day,온도토양,온도급액,온도배액,EC비율,pH비율,과중,당도,산도
0,2.876224,3.596165,5.682269,6.034272,433.472317,19.322963,1499.000000,17.682453,19.214050,19.631269,1.250308,1.061948,205.0,4.1,4.0
1,2.879453,3.502419,5.696574,6.020120,403.647635,19.679797,1560.000000,17.685968,19.482957,19.937158,1.216349,1.056797,210.0,4.7,4.0
2,2.784352,3.622177,5.660515,5.894754,553.805970,17.119403,1199.000000,17.685057,17.726729,18.103952,1.300905,1.041381,205.0,4.7,4.0
3,2.851406,3.603188,5.680561,6.048260,428.710361,19.725948,1412.000000,18.042475,19.559605,19.986085,1.263653,1.064729,206.0,4.7,4.5
4,2.946274,4.268803,5.676011,5.795994,504.994318,27.460341,487.173126,20.118731,23.217148,23.307148,1.448882,1.021139,223.0,4.3,4.0


In [20]:
X = tom_mul[tom_mul.columns[:-3]]
y = tom_mul[tom_mul.columns[-3:]]
scaler = StandardScaler()
tom_mul_X_scaler = scaler.fit_transform(X)
tom_mul_X_scaler = pd.DataFrame(tom_mul_X_scaler, columns = tom_mul.columns[:-3])
tom_mul_scale = pd.concat([tom_mul_X_scaler, y], axis = 1)

### SVM

In [21]:
# multi_imputer
X = tom_mul_scale[tom_mul_scale.columns[:-3]]
y_w = tom_mul_scale[tom_mul_scale.columns[-3]]
y_s = tom_mul_scale[tom_mul_scale.columns[-2]]
y_p = tom_mul_scale[tom_mul_scale.columns[-1]]

# 데이터 분리
X_train, X_test, y_train_w, y_test_w = train_test_split(X, y_w, 
                                                        random_state=0,
                                                       test_size=0.2)
X_train, X_test, y_train_s, y_test_s = train_test_split(X, y_s, 
                                                        random_state=0,
                                                       test_size=0.2)
X_train, X_test, y_train_p, y_test_p = train_test_split(X, y_p, 
                                                        random_state=0,
                                                       test_size=0.2)


# 파라미터
params = {"C" : [round(100*0.1**i, i) for i in range(8)],
         "gamma" : ['scale', 'auto'],
         "kernel" : ['linear', 'poly', 'rbf']}

# 과중
grid_svr = GridSearchCV(SVR(), params, cv = 5)
grid_svr.fit(X_train, y_train_w)

pred = grid_svr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test_w, pred))

print("<과중>")
print("파라미터 : ", grid_svr.best_params_)
print("훈련 점수 : ", grid_svr.best_score_)
print("테스트 점수: ", grid_svr.score(X_test, y_test_w))
print("rmse :", rmse)
print("----------------------------------------------------")

# 당도
grid_svr = GridSearchCV(SVR(), params, cv = 5)
grid_svr.fit(X_train, y_train_s)

pred = grid_svr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test_s, pred))

print("<당도>")
print("파라미터 : ", grid_svr.best_params_)
print("훈련 점수 : ", grid_svr.best_score_)
print("테스트 점수: ", grid_svr.score(X_test, y_test_s))
print("rmse :", rmse)
print("----------------------------------------------------")

# 산도
grid_svr = GridSearchCV(SVR(), params, cv = 5)
grid_svr.fit(X_train, y_train_p)

pred = grid_svr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test_p, pred))

print("<산도>")
print("파라미터 : ", grid_svr.best_params_)
print("훈련 점수 : ", grid_svr.best_score_)
print("테스트 점수: ", grid_svr.score(X_test, y_test_p))
print("rmse :", rmse)

<과중>
파라미터 :  {'C': 100.0, 'gamma': 'scale', 'kernel': 'linear'}
훈련 점수 :  0.7131041806782663
테스트 점수:  0.5820066623870437
rmse : 42.958067763412615
----------------------------------------------------
<당도>
파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}
훈련 점수 :  0.6327789824320197
테스트 점수:  0.6638000679886102
rmse : 0.27039525651805263
----------------------------------------------------
<산도>
파라미터 :  {'C': 100.0, 'gamma': 'scale', 'kernel': 'linear'}
훈련 점수 :  0.2144280407328388
테스트 점수:  0.05262996447710977
rmse : 0.1722410849558781


In [22]:
"""
# seed 0
<당도>
파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}
훈련 점수 :  0.7850628155052629
테스트 점수:  0.7690867672848773
rmse : 0.22409113105398093

# seed 40
<당도>
파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}
훈련 점수 :  0.7349473812550688
테스트 점수:  0.7839290445450481
rmse : 0.21676963890050632

# seed 41
<당도>
파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}
훈련 점수 :  0.8138370691756458
테스트 점수:  0.8533972494970379
rmse : 0.17855471266710274

# seed 42
<당도>
파라미터 :  {'C': 10.0, 'gamma': 'auto', 'kernel': 'rbf'}
훈련 점수 :  0.6762631754728139
테스트 점수:  0.7299809161828651
rmse : 0.24232459058061243

# seed 99
<당도>
파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}
훈련 점수 :  0.8589362297318257
테스트 점수:  0.8492964425401741
rmse : 0.18103477646452004

# seed 100
<당도>
파라미터 :  {'C': 1.0, 'gamma': 'scale', 'kernel': 'linear'}
훈련 점수 :  0.8756088807297685
테스트 점수:  0.8827656681157013
rmse : 0.1596715952761799

# seed 101
<당도>
파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}
훈련 점수 :  0.553171403465352
테스트 점수:  0.564126962034226
rmse : 0.30787913711221976
"""




"\n# seed 0\n<당도>\n파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}\n훈련 점수 :  0.7850628155052629\n테스트 점수:  0.7690867672848773\nrmse : 0.22409113105398093\n\n# seed 40\n<당도>\n파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}\n훈련 점수 :  0.7349473812550688\n테스트 점수:  0.7839290445450481\nrmse : 0.21676963890050632\n\n# seed 41\n<당도>\n파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}\n훈련 점수 :  0.8138370691756458\n테스트 점수:  0.8533972494970379\nrmse : 0.17855471266710274\n\n# seed 42\n<당도>\n파라미터 :  {'C': 10.0, 'gamma': 'auto', 'kernel': 'rbf'}\n훈련 점수 :  0.6762631754728139\n테스트 점수:  0.7299809161828651\nrmse : 0.24232459058061243\n\n# seed 99\n<당도>\n파라미터 :  {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}\n훈련 점수 :  0.8589362297318257\n테스트 점수:  0.8492964425401741\nrmse : 0.18103477646452004\n\n# seed 100\n<당도>\n파라미터 :  {'C': 1.0, 'gamma': 'scale', 'kernel': 'linear'}\n훈련 점수 :  0.8756088807297685\n테스트 점수:  0.8827656681157013\nrmse : 0.1596715952761799\n\n# seed 101\n<당도>\n파라미터 :  {'C':